In [ ]:
import codecs
import numpy as np

In [ ]:

barna = codecs.open('data/allebarna_v2.txt', encoding='utf8').readlines()

In [ ]:
kaizer = codecs.open('data/kaizers.txt', encoding='utf8').readlines()

In [ ]:
barna = [vits for vits in barna if vits[0:4]=='Alle']

In [ ]:
len(barna)

In [ ]:
len(kaizer)

In [ ]:
kaizer[:10]

In [ ]:
barna[:10]

In [ ]:
alle = kaizer + barna

In [ ]:
len(alle)

In [ ]:
kaizerbarna = [sent.replace("\n", "").lower() for sent in alle]

In [ ]:
kaizerbarna[:10]

In [ ]:
seqs = []
seq_length = 10

for line in kaizerbarna:
    if len(line)<=10:
        seqs.append(line)
    else:
        for i in range(seq_length, len(line)):
            seqs.append(line[i-seq_length:i+1])


In [ ]:
seqs[:10]

In [ ]:
len(seqs)

In [ ]:
len(set(''.join(seqs)))

In [ ]:
realseqs = [seq for seq in seqs if len(seq)==11]

In [ ]:
len(realseqs)

In [ ]:
chars = sorted(list(set(''.join(realseqs))))
mapping = dict((c, i) for i, c in enumerate(chars))

In [ ]:
mapping

In [ ]:
sequences = list()
for line in realseqs:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

In [ ]:
sequences[0]

In [ ]:
np_sequences = np.array(sequences)

In [ ]:
np_sequences

In [ ]:
X, y = np_sequences[:,:-1], np_sequences[:,-1]

In [ ]:
sequences[0]

In [ ]:
import numpy as np
from keras.utils import to_categorical
vocab_size = len(set(''.join(seqs)))

sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=2, verbose=1)

In [ ]:

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text



In [ ]:
print(generate_seq(model, mapping, 10, 'håndtverk', 150))